# Exeuctive Orders Raw Text
## Extract based on pdf url

## 0- Preparation

In [ ]:
import pandas as pd
import requests
import os
import io
import PyPDF2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install requests PyPDF2 pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


## 1- Text Data Extraction

In [ ]:
# Function to download and extract text from a PDF
def scrape(url, folder_path):
    try:
        # Get the PDF filename from the URL
        filename = url.split("/")[-1]

        # Send a GET request to download the PDF
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful

        # Save the PDF to the specified folder in Google Drive
        pdf_file_path = os.path.join(folder_path, filename)
        with open(pdf_file_path, 'wb') as pdf_file:
            pdf_file.write(response.content)

        # Extract text using PyPDF2
        text = ''
        with io.BytesIO(response.content) as pdf_stream:
            reader = PyPDF2.PdfReader(pdf_stream)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()  # Extract text from each page

        return text
    except Exception as e:
        return f"Error scraping {url}: {str(e)}"
    

In [ ]:
# Create a folder in Google Drive to store the PDFs
folder_path = '/content/drive/My Drive/Executive Orders/'

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Load the dataset and select only the first 30 rows and the 'pdf_url' column
csv_path = '/content/drive/MyDrive/documents_of_type_presidential_document_and_of_presidential_document_type_executive_order.csv'
data = pd.read_csv(csv_path)

# Select the first 30 rows of the 'pdf_url' column
# for test
# data_subset = data.head(50)

# Apply the function to download, save, and extract PDF text for each URL
# data_subset['pdf_text'] = data_subset['pdf_url'].apply(lambda url: scrape_and_save_pdf(url, folder_path))
data['pdf_text'] = data['pdf_url'].apply(lambda url: scrape(url, folder_path))
# Merge the new column back to the original dataframe
# data.loc[data_subset.index, 'pdf_text'] = data_subset['pdf_text']
data.loc[data.index, 'pdf_text'] = data['pdf_text']

new_csv_path = '/content/drive/My Drive/Executive Orders/presidential_document_raw_text.csv'
data.to_csv(new_csv_path, index=False)

print(f"PDFs saved to Google Drive folder: {folder_path}")
print(f"Original CSV updated and renamed to: {new_csv_path}")



PDFs saved to Google Drive folder: /content/drive/My Drive/Executive Orders/
Original CSV updated and renamed to: /content/drive/My Drive/Executive Orders/presidential_document_raw_text.csv
